In [8]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import pocketsphinx

### Bag 0 : Trial n Error

In [ ]:
data

### Bag 1 : Classification Hijaiyah

In [6]:



# Load and preprocess dataset
# ... (data loading and preprocessing steps)

# Build the CNN model
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(28, 28, 1)),
    MaxPooling2D((2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Flatten(),
    Dense(128, activation='relu'),
    Dense(28, activation='softmax')  # Assuming 28 classes for Hijaiyah letters
])

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Data augmentation
datagen = ImageDataGenerator(
    rotation_range=10,
    zoom_range=0.1,
    width_shift_range=0.1,
    height_shift_range=0.1
)

# Train the model
# train_images, train_labels should be preprocessed datasets
model.fit(datagen.flow(train_images, train_labels, batch_size=32), epochs=10, validation_data=(val_images, val_labels))

# Save the model
model.save('hijaiyah_model.h5')


### Bag 2 : Recognize Doa Speech